# 📚 Treinamento de Modelo NER com spaCy

## 📑 Análise de Projetos de Lei e Consultas Legislativas da Câmara dos Deputados

Este notebook apresenta o processo de **treinamento** e **avaliação** de um modelo de **Reconhecimento de Entidades Nomeadas (NER)** utilizando a biblioteca **spaCy**.
  
O conjunto de dados utilizado é composto por **projetos de lei** e **consultas legislativas** da **Câmara dos Deputados do Brasil**, extraídos do corpus **[UlyssesNER-Br: A Corpus of Brazilian Legislative Documents for Named Entity Recognition](https://github.com/Convenio-Camara-dos-Deputados/ulyssesner-br-propor/tree/main/PL-corpus_v2/ulysses_categories/holdout)**.


## ⚙️ Carregamento e Pré-processamento

O repositório definido para o desenvolvimento possui a divisão *holdout* em três partes: **train**, **dev** e **test**. Estes dados foram extraídos diretamente do repositório via `requests` e, na sequência, passaram por um pipeline de pré-processamento que garante a estrutura correta para o treinamento do modelo de NER.

Cada etapa tem sua função específica:

- **Carregar em formato `Dataset`**: Permite manipular os dados de forma estruturada e eficiente, facilitando a aplicação de funções de transformação, filtragem e mapeamento dos exemplos.

- **Mapear rótulos (BIO) para IDs**: Converte rótulos de texto para valores numéricos, que são necessários para o modelo aprender as classes de forma otimizada e sem ambiguidades.

- **Converter sequência BIO em spans de entidades**: Transforma a sequência de rótulos em informações de posição no texto, delimitando onde cada entidade realmente começa e termina.

- **Unir rótulos B- e I-**: Garante que cada entidade seja representada como um único bloco contínuo no texto.

- **Preparar lista final para treino**: Formata os dados em pares `(texto, entidades)` prontos para serem transformados em objetos do spaCy.

- **Converter para `DocBin` do spaCy**: Gera um arquivo binário otimizado que o spaCy utiliza para treinar o modelo.

Essas etapas asseguram que os dados estejam prontos e padronizados para o treinamento do modelo com spaCy.

In [ ]:
import requests, json
from datasets import Dataset
import numpy as np
from datasets import load_metric

In [ ]:
base_url = "https://raw.githubusercontent.com/Convenio-Camara-dos-Deputados/ulyssesner-br-propor/main/PL-corpus_v2/ulysses_categories/holdout/"
urls = {
    "train": base_url + "train.json",
    "dev": base_url + "dev.json",
    "test": base_url + "test.json"
}

In [ ]:
def load_data(url):
    r = requests.get(url)
    return json.loads(r.text)

In [ ]:
train_data = load_data(urls["train"])
dev_data = load_data(urls["dev"])
test_data = load_data(urls["test"])

In [ ]:
def prepare_dataset(data):
    return {
        "tokens": [ex["tokens"] for ex in data],
        "ner_tags": [ex["ner_tokens"] for ex in data]
    }

In [ ]:
train_dict = prepare_dataset(train_data)
dev_dict = prepare_dataset(dev_data)
test_dict = prepare_dataset(test_data)

In [ ]:
train_dataset = Dataset.from_dict(train_dict)
dev_dataset = Dataset.from_dict(dev_dict)
test_dataset = Dataset.from_dict(test_dict)

In [ ]:
unique_labels = list(set(label for doc in train_dict["ner_tags"] for label in doc))
unique_labels.sort()

label_to_id = {l: i for i, l in enumerate(unique_labels)}
id_to_label = {i: l for i, l in enumerate(unique_labels)}

print("Labels:", unique_labels)

Labels: ['B-DATA', 'B-EVENTO', 'B-FUNDAMENTO', 'B-LOCAL', 'B-ORGANIZACAO', 'B-PESSOA', 'B-PRODUTODELEI', 'I-DATA', 'I-EVENTO', 'I-FUNDAMENTO', 'I-LOCAL', 'I-ORGANIZACAO', 'I-PESSOA', 'I-PRODUTODELEI', 'O']


In [ ]:
def convert_to_entity_dicts(data: dict) -> tuple[list[dict], str]:
    """
    Converte tokens + ner_tags em spans (start, end, label, text).
    """
    tokens = data["tokens"]
    ner_tags = data["ner_tags"]

    # Se as tags forem strings, mantém; se forem IDs, converte:
    if isinstance(ner_tags[0], int):
        ner_tags = [id_to_label[tag_id] for tag_id in ner_tags]

    entity_dicts = []
    original_text = " ".join(tokens)

    i = 0
    while i < len(tokens):
        if ner_tags[i] != "O":
            label = ner_tags[i]
            start_index = len(" ".join(tokens[:i])) + int(i > 0)
            start_token_idx = i

            while i < len(tokens) and ner_tags[i] == label:
                i += 1
            end_index = len(" ".join(tokens[:i]))
            entity_text = " ".join(tokens[start_token_idx:i])

            entity_dicts.append(
                {
                    "start": start_index,
                    "end": end_index,
                    "label": label,
                    "text": entity_text,
                }
            )
        else:
            i += 1

    return entity_dicts, original_text


def merge_entities_of_same_type(text: str, entity_spans: list[dict]) -> list[dict]:
    """
    Junta B- e I- em um único span por tipo.
    """
    merged_entity_spans = []
    i = 0
    while i < len(entity_spans):
        if entity_spans[i]["label"].startswith("B-"):
            start_idx = entity_spans[i]["start"]
            end_idx = entity_spans[i]["end"]
            entity_type = entity_spans[i]["label"][2:]
            entity_text = entity_spans[i]["text"]
            j = i + 1

            while (
                j < len(entity_spans)
                and entity_spans[j]["label"] == f"I-{entity_type}"
            ):
                end_idx = entity_spans[j]["end"]
                entity_text = text[start_idx:end_idx]
                j += 1

            merged_entity_spans.append(
                {
                    "start": start_idx,
                    "end": end_idx,
                    "label": entity_type,
                    "text": entity_text,
                }
            )
            i = j
        else:
            label = entity_spans[i]["label"]
            if label.startswith(("B-", "I-")):
                label = label[2:]
            merged_entity_spans.append(
                {
                    "start": entity_spans[i]["start"],
                    "end": entity_spans[i]["end"],
                    "label": label,
                    "text": entity_spans[i]["text"],
                }
            )
            i += 1

    return merged_entity_spans


In [ ]:
train_dataset[0]

{'tokens': ['sala', 'das', 'sessões', ',', 'em', 'de', 'de', '2019', '.'],
 'ner_tags': ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-DATA', 'O']}

In [ ]:
ner_dict, text = convert_to_entity_dicts(train_dataset[0])
ner_dict, text

([{'start': 28, 'end': 32, 'label': 'B-DATA', 'text': '2019'}],
 'sala das sessões , em de de 2019 .')

In [ ]:
ner_dict = merge_entities_of_same_type(text, ner_dict)
ner_dict

[{'start': 28, 'end': 32, 'label': 'DATA', 'text': '2019'}]

In [ ]:
from spacy import displacy
from termcolor import colored

def render_named_entities_terminal(text, ner_dict):
    """
    Imprime o texto com entidades destacadas no terminal.
    """
    spans = sorted(ner_dict, key=lambda x: x['start'])
    output = ""
    last_idx = 0

    for ent in spans:
        output += text[last_idx:ent['start']]
        output += colored(text[ent['start']:ent['end']], 'green')
        output += f"[{ent['label']}]"
        last_idx = ent['end']

    output += text[last_idx:]
    print(output)


In [ ]:
render_named_entities(text, ner_dict)

In [ ]:
def prepare_data_for_training(dataset, id2label):
    """
    Converte Dataset HuggingFace para lista de (text, entities).
    Usa suas funções convert_to_entity_dicts + merge_entities_of_same_type.

    Args:
        dataset: Dataset do HuggingFace.
        id2label: dict mapeando ID → label (BIO).

    Returns:
        list[tuple[str, list[dict]]]: Cada item é (text, entities)
    """
    prepared_data = []

    for data_item in dataset:
        entity_dicts, text = convert_to_entity_dicts(data_item)
        entity_dicts = merge_entities_of_same_type(text, entity_dicts)
        prepared_data.append((text, entity_dicts))

    return prepared_data


In [ ]:
training_data = prepare_data_for_training(train_dataset, id_to_label)
validation_data = prepare_data_for_training(dev_dataset, id_to_label)
test_data = prepare_data_for_training(test_dataset, id_to_label)


In [ ]:
import spacy
from spacy.tokens import DocBin

# Load a new blank spaCy model for Portuguese
nlp = spacy.blank("pt")


def convert_to_spacy(data_list):
    """
    Converts a list of texts and their annotations to a spaCy DocBin object.

    Args:
        data_list (list): A list of tuples containing the text and annotations.
            Each annotation should be a dictionary with keys "start", "end", "label", and "text".

    Returns:
        DocBin: A spaCy DocBin object containing the labeled texts.
    """
    # Create a DocBin object to store the processed documents
    db = DocBin()

    # Iterate over each text and its corresponding annotations in the data list
    for text, annot in data_list:
        # Create a spaCy Doc object from the text
        doc = nlp.make_doc(text)
        ents = []
        seen_tokens = set()

        # Iterate over each annotation dictionary
        for entity in annot:
            start = entity["start"]
            end = entity["end"]
            label = entity["label"]

            # Create a span for the entity using character indexes.
            span = doc.char_span(start, end, label=label, alignment_mode="contract")

            # If the span is None, the character span does not align with token boundaries.
            if span is None:
                print(
                    f"Skipping entity [{start}, {end}, {label}] in the following text because "
                    f"the character span '{text[start:end]}' does not align with token boundaries:\n\n{text}\n"
                )
            else:
                # Check for overlapping tokens
                if any(token.i in seen_tokens for token in span):
                    continue
                else:
                    # Add the span to the list of entities
                    ents.append(span)
                    seen_tokens.update(token.i for token in span)

        # Assign the list of entities to the doc.ents attribute
        doc.ents = ents

        # Add the processed document to the DocBin object
        db.add(doc)

    # Return the DocBin object containing all the processed documents
    return db

In [ ]:
# Example usage:
# Assuming training_data, validation_data, and test_data are defined somewhere,
# and each annotation is now a dict with keys "start", "end", "label", and "text".
db_train = convert_to_spacy(training_data)
db_valid = convert_to_spacy(validation_data)
db_test = convert_to_spacy(test_data)

In [ ]:

from pathlib import Path

output_dir = Path("./outputs/spacy")  # output directory
output_dir.mkdir(parents=True, exist_ok=True)  # create the output directory

In [ ]:
# Save the data to disk
db_train.to_disk("./outputs/spacy/train.spacy")
db_valid.to_disk("./outputs/spacy/valid.spacy")
db_test.to_disk("./outputs/spacy/test.spacy")

In [ ]:
# If we want to load it later
from spacy.tokens import DocBin

db_train = DocBin().from_disk("./outputs/spacy/train.spacy")
db_valid = DocBin().from_disk("./outputs/spacy/valid.spacy")
db_test = DocBin().from_disk("./outputs/spacy/test.spacy")
len(db_train), len(db_valid), len(db_test)

(1760, 140, 592)

# ⚙️ Criação da Configuração e Treinamento do Modelo

Após preparar os dados no formato `DocBin`, é necessário criar o arquivo de configuração do spaCy, que define os parâmetros de linguagem, pipeline, otimização e caminhos de entrada/saída do modelo.  

O comando `spacy init config` gera esse arquivo `config.cfg` automaticamente, ajustado para o idioma **português**, com o pipeline `ner` (Reconhecimento de Entidades Nomeadas) e otimizações para maior eficiência de processamento.

Em seguida, o comando `spacy train` executa o treinamento do modelo, utilizando os dados de treino e validação convertidos anteriormente.

In [ ]:
# Creating the config file
! python -m spacy init config "./outputs/spacy/config.cfg" --lang pt --pipeline ner --optimize efficiency --force

⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: pt
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
outputs/spacy/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
# Training

! python -m spacy train "./outputs/spacy/config.cfg" \
                        --output "./outputs/spacy" \
                        --paths.train "./outputs/spacy/train.spacy" \
                        --paths.dev "./outputs/spacy/valid.spacy"

ℹ Saving to output directory: outputs/spacy
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     29.50    0.00    0.00    0.00    0.00
  0     200       1213.82   1641.53   11.22   20.37    7.75    0.11
  0     400        601.88   1104.12   28.31   40.26   21.83    0.28
  0     600        248.62   1168.71   46.22   62.65   36.62    0.46
  1     800       1371.28   1314.33   48.31   60.64   40.14    0.48
  1    1000        479.88   1302.92   55.87   65.71   48.59    0.56
  2    1200       1814.55   1223.75   58.17   66.97   51.41    0.58
  3    1400        889.26   1278.00   66.42   70.63   62.68    0.66
  4    1600       1047.0

# ✅ Avaliação do Modelo

Após o término do treinamento, o modelo treinado é carregado a partir do diretório `model-best`, que contém a versão com o melhor desempenho validado automaticamente pelo spaCy.  

Para avaliar a qualidade do modelo, utiliza-se o conjunto de **teste**, também convertido em formato `DocBin`. Cada exemplo do conjunto de teste é comparado com as predições do modelo por meio da classe `Example`, que alinha as entidades previstas com as entidades anotadas.

O resultado da avaliação inclui métricas de desempenho como **precision**, **recall** e **F1-score** para cada tipo de entidade, além de métricas gerais do modelo.


In [ ]:
# Evaluation
from spacy.training.example import Example

nlp_ner = spacy.load("./outputs/spacy/model-best")
db_test = DocBin().from_disk("./outputs/spacy/test.spacy")


examples = []
for doc in db_test.get_docs(nlp_ner.vocab):
    examples.append(Example(nlp_ner.make_doc(doc.text), doc))

results_spacy = nlp_ner.evaluate(examples)
results_spacy

{'token_acc': 1.0,
 'token_p': 1.0,
 'token_r': 1.0,
 'token_f': 1.0,
 'ents_p': 0.7594108019639935,
 'ents_r': 0.651685393258427,
 'ents_f': 0.7014361300075586,
 'ents_per_type': {'PESSOA': {'p': 0.8256880733944955,
   'r': 0.6666666666666666,
   'f': 0.7377049180327869},
  'PRODUTODELEI': {'p': 0.6595744680851063,
   'r': 0.49206349206349204,
   'f': 0.5636363636363635},
  'ORGANIZACAO': {'p': 0.6972477064220184,
   'r': 0.6129032258064516,
   'f': 0.6523605150214592},
  'FUNDAMENTO': {'p': 0.6987951807228916,
   'r': 0.7945205479452054,
   'f': 0.7435897435897436},
  'DATA': {'p': 0.8860759493670886,
   'r': 0.8333333333333334,
   'f': 0.8588957055214723},
  'LOCAL': {'p': 0.801980198019802,
   'r': 0.5192307692307693,
   'f': 0.6303501945525292},
  'EVENTO': {'p': 0.0, 'r': 0.0, 'f': 0.0}},
 'speed': 12131.672033139972}

In [ ]:
import pandas as pd

pd.DataFrame(results_spacy["ents_per_type"]).T

,p,r,f
PESSOA,0.825688,0.666667,0.737705
PRODUTODELEI,0.659574,0.492063,0.563636
ORGANIZACAO,0.697248,0.612903,0.652361
FUNDAMENTO,0.698795,0.794521,0.743590
DATA,0.886076,0.833333,0.858896
LOCAL,0.801980,0.519231,0.630350
EVENTO,0.000000,0.000000,0.000000


## 🔍 Análise dos Resultados de Avaliação

- **Token accuracy** (`token_acc`): O valor 1.0 mostra que o modelo segmentou os tokens exatamente como esperado.
  - Isso é típico quando se utiliza a tokenização padrão do spaCy em textos já limpos e bem formatados.

- **Precision (`ents_p`) = 75.9%**: Dos spans de entidades previstos pelo modelo, aproximadamente 76% foram corretos — ou seja, a maior parte dos spans detectados realmente corresponde a entidades anotadas.

- **Recall (`ents_r`) = 65.1%**: O modelo conseguiu recuperar cerca de 65% de todas as entidades reais presentes no conjunto de teste. Isso mostra que ainda há entidades que não foram reconhecidas (falsos negativos).

- **F1-score (`ents_f`) = 70.1%**: A pontuação F1 combina **Precision** e **Recall**, fornecendo uma visão equilibrada da qualidade do modelo. Um valor acima de 70% é um bom ponto de partida, considerando a complexidade de textos legislativos.

### 📊 Desempenho por Tipo de Entidade

- **PESSOA**: Bom desempenho (F1-score ~73,7%) com alta Precision (82,5%), mas Recall moderado (66,6%), sugerindo que o modelo reconhece bem nomes, mas ainda deixa escapar alguns.
- **PRODUTODELEI**: Desempenho mais baixo (F1-score ~56%), indicando maior dificuldade em identificar corretamente menções a projetos de lei ou documentos legislativos.
- **ORGANIZACAO**: F1-score de ~65%, com equilíbrio entre Precision e Recall, mas ainda com espaço para melhorias.
- **FUNDAMENTO**: F1-score de ~74%, destacando que o modelo identifica bem fundamentos legais ou citações de normas.
- **DATA**: Melhor desempenho individual (F1-score ~85%), mostrando que datas são relativamente fáceis de capturar no contexto legislativo.
- **LOCAL**: F1-score de ~63%, indicando que a detecção de localidades ainda tem variações — possivelmente por ambiguidades ou contextos complexos.
- **EVENTO**: Resultado zerado (F1-score = 0), sugerindo que o conjunto de teste não contém exemplos marcados dessa classe, ou que o modelo não aprendeu a capturá-los.

### ✅ Considerações Finais

Em resumo, o modelo atinge uma qualidade satisfatória para uma primeira versão, mas ainda há espaço para melhorias:
- Aumentar o volume de exemplos para classes com baixo Recall.
- Refinar as anotações para reduzir inconsistências.
- Explorar hiperparâmetros ou modelos base maiores, se necessário.
